<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [24]:
%matplotlib inline
%load_ext autoreload
%autoreload 4
%autosave 120

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 120 seconds


In [25]:
from fastai.io import *
from fastai.structured import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from pandas_summary import DataFrameSummary
from IPython.display import display
from sklearn import metrics
import feather
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
import warnings
import gc
from sklearn.linear_model import Ridge
import xgboost as xgb

In [26]:
train_df2 = feather.read_dataframe('train_df_alpha')
test_df2 = feather.read_dataframe('test_df_alpha')

In [27]:
for df in [train_df2, test_df2]:
    for f in ['purchase_date_max','purchase_date_min','purchase_date_max_old',\
                     'purchase_date_min_old', 'observation_date_old']:
        df[f] = df[f].astype(np.int64) * 1e-9

In [28]:
cols_excluded = ['purchase_date_max', 'purchase_date_max_old', 'card_id', 'first_active_month',
                 'target','outliers','card_id_size', 'card_id_size_old', 
                 'purchase_date_min', 'purchase_date_min_old','first_active_monthYear',
                 'first_active_monthMonth',
                 'first_active_monthWeek',
                 'first_active_monthDay',
                 'first_active_monthDayofweek',
                 'first_active_monthDayofyear',
                 'first_active_monthIs_month_end',
                 'first_active_monthIs_month_start',
                 'first_active_monthIs_quarter_end',
                 'first_active_monthIs_quarter_start',
                 'first_active_monthIs_year_end',
                 'Black_Friday_2017_mean',
                 'amount_month_ratio_max',
                 'purchase_Month_mean_old',
                 'purchase_amount_total_max',
                 'first_active_monthIs_year_start']

In [29]:
cols_included = ['feature_1','feature_2','feature_3','transactions_count','subsector_id_nunique','merchant_id_nunique','merchant_category_id_nunique','purchase_Month_mean','purchase_Month_min','purchase_Month_max','purchase_Week_nunique','purchase_Week_mean','purchase_Week_min','purchase_Week_max','purchase_Dayofweek_mean','purchase_Dayofweek_min','purchase_Dayofweek_max','purchase_Day_nunique','purchase_Day_mean','purchase_Day_min','purchase_Day_max','purchase_Hour_nunique','purchase_Hour_mean','purchase_Hour_min','purchase_Hour_max','purchase_amount_sum','purchase_amount_max','purchase_amount_min','purchase_amount_mean','purchase_amount_var','purchase_amount_skew','installments_sum','installments_max','installments_mean','installments_var','installments_skew','month_lag_max','month_lag_min','month_lag_mean','month_lag_var','month_lag_skew','month_diff_mean','month_diff_var','month_diff_skew','purchased_on_weekend_mean','category_1_mean','category_2_mean','category_3_mean','card_id_count','price_mean','price_max','price_min','price_var','Christmas_Day_2017_mean','Children_day_2017_mean','Black_Friday_2017_mean','Mothers_Day_2018_mean','duration_mean','duration_min','duration_max','duration_var','duration_skew','amount_month_ratio_mean','amount_month_ratio_min','amount_month_ratio_max','amount_month_ratio_var','amount_month_ratio_skew','category_2_mean_mean','category_3_mean_mean','purchase_date_diff','purchase_date_average','purchase_date_uptonow','purchase_date_uptomin','transactions_count_old','subsector_id_nunique_old','merchant_id_nunique_old','merchant_category_id_nunique_old','purchase_Month_nunique','purchase_Month_mean_old','purchase_Month_min_old','purchase_Month_max_old','purchase_Week_nunique_old','purchase_Week_mean_old','purchase_Week_min_old','purchase_Week_max_old','purchase_Dayofweek_mean_old','purchase_Day_nunique_old','purchase_Day_mean_old','purchase_Day_min_old','purchase_Hour_nunique_old','purchase_Hour_mean_old','purchase_Hour_min_old','purchase_Hour_max_old','purchase_amount_sum_old','purchase_amount_max_old','purchase_amount_min_old','purchase_amount_mean_old','purchase_amount_var_old','purchase_amount_skew_old','installments_sum_old','installments_max_old','installments_mean_old','installments_var_old','installments_skew_old','month_lag_max_old','month_lag_min_old','month_lag_mean_old','month_lag_var_old','month_lag_skew_old','month_diff_max','month_diff_min','month_diff_mean_old','month_diff_var_old','month_diff_skew_old','authorized_flag_mean','purchased_on_weekend_mean_old','category_1_mean_old','category_2_mean_old','category_3_mean_old','card_id_count_old','price_sum','price_mean_old','price_max_old','price_min_old','price_var_old','Christmas_Day_2017_mean_old','Mothers_Day_2017_mean','fathers_day_2017_mean','Children_day_2017_mean_old','Valentine_Day_2017_mean','Black_Friday_2017_mean_old','Mothers_Day_2018_mean_old','duration_mean_old','duration_min_old','duration_max_old','duration_var_old','duration_skew_old','amount_month_ratio_mean_old','amount_month_ratio_min_old','amount_month_ratio_max_old','amount_month_ratio_var_old','amount_month_ratio_skew_old','category_2_mean_mean_old','category_3_mean_mean_old','purchase_date_diff_old','purchase_date_average_old','purchase_date_uptonow_old','purchase_date_uptomin_old','quarter','observed_elapsed_time','days_feature1','days_feature2','days_feature3','days_feature1_ratio','days_feature2_ratio','days_feature3_ratio','feature_sum','feature_mean','feature_max','feature_min','feature_var','card_id_total','card_id_count_total','card_id_count_ratio','purchase_amount_total','purchase_amount_total_mean','purchase_amount_total_max','purchase_amount_total_min','purchase_amount_sum_ratio','hist_first_buy','new_first_buy','hist_last_buy','new_last_buy','month_diff_ratio','installments_total','installments_ratio','price_total','CLV','CLV_old','CLV_ratio']

In [30]:
df_train_columns = [c for c in cols_included if c not in cols_excluded]

In [31]:
len(df_train_columns)

176

In [32]:
extra_cols = ['merchant_address_id_nunique', 'merchant_rating_nunique']

In [33]:
df_train_columns = df_train_columns + extra_cols

In [34]:
len(df_train_columns)

178

In [35]:
target = train_df2['target']

In [36]:
del train_df2['target']

In [37]:
len(df_train_columns)

178

In [38]:
xgb_params = {
    'objective': 'reg:linear', 
    'eval_metric': 'rmse', 
    'silent': True, 
    'booster': 'gbtree', 
    'n_jobs': 4, 
    'n_estimators': 20000, 
    'grow_policy': 'lossguide', 
    'max_depth': 12, 
    'seed': 538, 
    'colsample_bylevel': 0.9, 
    'colsample_bytree': 0.8, 
    'gamma': 0.0001, 
    'learning_rate': 0.006150886706231842, 
    'max_bin': 128, 
    'max_leaves': 47, 
    'min_child_weight': 40, 
    'reg_alpha': 10.0, 
    'reg_lambda': 10.0, 
    'silent': True,
    'eta': 0.005,
    'subsample': 0.9
}

# xgb_params = {'eta': 0.005, 'max_depth': 10, 'subsample': 0.8, 'colsample_bytree': 0.8, 
#           'objective': 'reg:linear', 'eval_metric': 'rmse', 'silent': True}


In [39]:
oof_xgb_3 = np.zeros(len(train_df2))
predictions_xgb_3 = np.zeros(len(test_df2))

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=15)


In [40]:
gc.collect()

240

In [41]:
%%time
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df2,train_df2['outliers'].values)):    
    print('-')
    print("Fold {}".format(fold_ + 1))
    trn_data = xgb.DMatrix(data=train_df2.iloc[trn_idx][df_train_columns], label=target.iloc[trn_idx])
    val_data = xgb.DMatrix(data=train_df2.iloc[val_idx][df_train_columns], label=target.iloc[val_idx])
    watchlist = [(trn_data, 'train'), (val_data, 'valid')]
    print("xgb " + str(fold_) + "-" * 50)
    num_round = 10000
    xgb_model = xgb.train(xgb_params, trn_data, num_round, watchlist, early_stopping_rounds=500, verbose_eval=1000)
    oof_xgb_3[val_idx] = xgb_model.predict(xgb.DMatrix(train_df2.iloc[val_idx][df_train_columns]), ntree_limit=xgb_model.best_ntree_limit+50)

    predictions_xgb_3 += xgb_model.predict(xgb.DMatrix(test_df2[df_train_columns]), ntree_limit=xgb_model.best_ntree_limit+50) / folds.n_splits
    
np.save('oof_xgb_3_2', oof_xgb_3)
np.save('predictions_xgb_3_2', predictions_xgb_3)
np.sqrt(mean_squared_error(target.values, oof_xgb_3))

-
Fold 1
xgb 0--------------------------------------------------
[0]	train-rmse:3.94728	valid-rmse:3.95376
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 500 rounds.
[1000]	train-rmse:2.87719	valid-rmse:3.65189
Stopping. Best iteration:
[682]	train-rmse:2.98069	valid-rmse:3.65003

-
Fold 2
xgb 1--------------------------------------------------
[0]	train-rmse:3.94767	valid-rmse:3.95308
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 500 rounds.
[1000]	train-rmse:2.88087	valid-rmse:3.66125
Stopping. Best iteration:
[1170]	train-rmse:2.83974	valid-rmse:3.66092

-
Fold 3
xgb 2--------------------------------------------------
[0]	train-rmse:3.95058	valid-rmse:3.94095
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 500 rounds.
[1000]	t

In [42]:
oof_xgb_3_2 = np.load('oof_xgb_3_2.npy')
predictions_xgb_3_2 = np.load('predictions_xgb_3_2.npy')

In [26]:
# oof_xgb_3 = np.load('oof_xgb_3.npy')
# predictions_xgb_3_2 = np.load('predictions_xgb_3_2.npy')

In [43]:
np.sqrt(mean_squared_error(target.values, oof_xgb_3_2))

3.649844028552147

In [23]:
# np.sqrt(mean_squared_error(target.values, oof_xgb_3))

In [14]:
submission_name = pd.to_datetime("today")

In [15]:
sub_df = pd.DataFrame({"card_id":test_df2["card_id"].values})
sub_df["target"] = predictions_xgb_3
sub_df.to_csv(f'xg_submission-{submission_name}.csv', index=False)

In [17]:
filename = f'xg_submission-{submission_name}.csv'

In [18]:
filename

'xg_submission-2019-02-21 11:18:00.066865.csv'

In [19]:
!kaggle competitions submit elo-merchant-category-recommendation -f 'xg_submission-2019-02-21 11:18:00.066865.csv' -m "xgboost with 180 cols"

100%|#######################################| 4.24M/4.24M [00:05<00:00, 798kB/s]
Successfully submitted to Elo Merchant Category Recommendation

In [32]:
df_train_columnsnse_cols = ['Black_Friday_2017_mean']

In [33]:
df_train_columns = [c for c in df_train_columns if c not in remove_cols]

In [44]:
len(df_train_columns)

178

In [45]:
%%time
n_fold = 5
param = {
        'task': 'train',
        'boosting': 'goss',
        'objective': 'regression',
        'metric': 'rmse',
        'learning_rate': 0.01,
        'subsample': 0.9855232997390695,
        'max_depth': 7,
        'top_rate': 0.9064148448434349,
        'num_leaves': 123,
        'min_child_weight': 41.9612869171337,
        'other_rate': 0.0721768246018207,
        'reg_alpha': 9.677537745007898,
        'colsample_bytree': 0.5665320670155495,
        'min_split_gain': 9.820197773625843,
        'reg_lambda': 8.2532317400459,
        'min_data_in_leaf': 21,
        'verbose': -1,
        'seed':int(2**n_fold),
        'bagging_seed':int(2**n_fold),
        'drop_seed':int(2**n_fold)
        }
folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=4590)
oof = np.zeros(len(train_df2))
predictions = np.zeros(len(test_df2))
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df2,train_df2['outliers'].values)):
    print("fold {}".format(fold_))
    trn_data = lgb.Dataset(train_df2.iloc[trn_idx][df_train_columns], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train_df2.iloc[val_idx][df_train_columns], label=target.iloc[val_idx])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 200)
    oof[val_idx] = clf.predict(train_df2.iloc[val_idx][df_train_columns], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = df_train_columns
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(test_df2[df_train_columns], num_iteration=clf.best_iteration) / folds.n_splits

print(np.sqrt(mean_squared_error(oof, target)), 'CV score')

fold 0
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.63714	valid_1's rmse: 3.70315
[200]	training's rmse: 3.54887	valid_1's rmse: 3.66357
[300]	training's rmse: 3.49898	valid_1's rmse: 3.65138
[400]	training's rmse: 3.46344	valid_1's rmse: 3.64693
[500]	training's rmse: 3.43717	valid_1's rmse: 3.64506
[600]	training's rmse: 3.41561	valid_1's rmse: 3.64369
[700]	training's rmse: 3.39611	valid_1's rmse: 3.64378
[800]	training's rmse: 3.37645	valid_1's rmse: 3.64344
[900]	training's rmse: 3.36122	valid_1's rmse: 3.64357
[1000]	training's rmse: 3.34474	valid_1's rmse: 3.64395
Early stopping, best iteration is:
[833]	training's rmse: 3.37103	valid_1's rmse: 3.64336
fold 1
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.63889	valid_1's rmse: 3.6947
[200]	training's rmse: 3.55007	valid_1's rmse: 3.65835
[300]	training's rmse: 3.49857	valid_1's rmse: 3.64708
[400]	training's rmse: 3.46521	valid_1's rmse: 3.64355


In [50]:
np.save('oof_lgbm', oof)
np.save('predictions_lgbm', predictions)

In [46]:
train_stack = np.vstack([oof, oof_xgb_3_2]).transpose()
test_stack = np.vstack([predictions, predictions_xgb_3_2]).transpose()

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=15)
oof_stacked = np.zeros(train_stack.shape[0])
predictions_stacked = np.zeros(test_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_stack, train_df2['outliers'].values)):
    print("fold n°{}".format(fold_))
    trn_data, trn_y = train_stack[trn_idx], target.iloc[trn_idx].values
    val_data, val_y = train_stack[val_idx], target.iloc[val_idx].values

    clf = Ridge(alpha=1)
    clf.fit(trn_data, trn_y)
    
    oof_stacked[val_idx] = clf.predict(val_data)
    predictions_stacked += clf.predict(test_stack) / folds.n_splits



np.sqrt(mean_squared_error(target.values, oof_stacked))

fold n°0
fold n°1
fold n°2
fold n°3
fold n°4


3.6416503351286953

1. Round 1
    - XGB - 3.653676915307743 - LB -3.696 - small number of params
    - LGB - 3.644578037274131
    - Stacked - 3.6427847564989353
1. Round 2
    - XGB - 3.649741559823745 - LB -3.696 - fine tuned params from kernel
    - LGB - 3.644578037274131
    - Stacked - 3.6422567644144066
2. Round 3 - added ['merchant_address_id_nunique', 'merchant_rating_nunique']
    - XGB - 3.649806733419173
    - LGB - 3.6445206627403186 CV score
    - Stacked - 3.6421515201025905
    - LGB - 3.644042133990217 CV score (without Black_Friday_2017_mean)
    - Stacked - 3.641770180430725 (without Black_Friday_2017_mean)
1. Round 4
    - XGB - 3.649844028552147
    - LGB - 3.6437856343297264 
    - Stacked - 3.6416503351286953

In [47]:
submission_name = pd.to_datetime("today")
sub_df = pd.DataFrame({"card_id":test_df2["card_id"].values})
sub_df["target"] = predictions_stacked
sub_df.to_csv(f'xg_submission_stacked_lgb-{submission_name}.csv', index=False)
filename = f'xg_submission_stacked_lgb-{submission_name}.csv'

In [48]:
filename

'xg_submission_stacked_lgb-2019-02-22 20:48:54.579689.csv'

In [31]:
!kaggle competitions submit elo-merchant-category-recommendation -f 'xg_submission_stacked_lgb-2019-02-21 13:41:48.181414.csv' -m "xgboost & lgbm stacked with 180 cols"

 72%|############################1          | 3.06M/4.24M [00:12<00:03, 310kB/s]^C
 79%|##############################6        | 3.33M/4.24M [00:13<00:03, 264kB/s]
User cancelled operation
